# 0. Подключение библиотеки для работы с кластером и сервисом

In [1]:
import client_lib

In [2]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /opt/conda/lib/python3.6/site-packages (20.0.2)


In [3]:
!pip install /home/jovyan/requirements.txt

ERROR: Invalid requirement: '/home/jovyan/requirements.txt'
Hint: It looks like a path. File '/home/jovyan/requirements.txt' does not exist.


# 1. Работа с S3

##### Сохранение кредов S3

In [4]:
from client_lib import save_aws_credentials
save_aws_credentials('b875178b-f537-4a1c-8977-7871bb9dc448-namespace', 'b63f3480-a700-48fb-85c5-b4a3b36f3713', 'BPAdF5E9CGAFv+rmBakFc+R0krVCHd/u4rPsqfG8')

'S3 Credentials "{\'result\': None, \'status\': \'ok\'}" created'

##### Перекладка с S3 на локальные диски кластера /home/jovyan необходимых файлов

In [5]:
client_lib.S3CopyJob("s3://9b8d0e97-6461-47cf-9507-21ede838c9ac-bucket/mnist.npz", "/home/jovyan/").submit()

'S3CopyJob "{\'name\': \'copying-job-mnxmr\', \'status\': \'ok\'}" created'

In [6]:
client_lib.S3CopyJob("s3://9b8d0e97-6461-47cf-9507-21ede838c9ac-bucket/requirements.txt", "/home/jovyan/").submit()

'S3CopyJob "{\'name\': \'copying-job-h67sz\', \'status\': \'ok\'}" created'

In [7]:
client_lib.S3CopyJob("s3://9b8d0e97-6461-47cf-9507-21ede838c9ac-bucket/tensorflow_mnist_estimator.py", "/home/jovyan/").submit()

'S3CopyJob "{\'name\': \'copying-job-29nn2\', \'status\': \'ok\'}" created'

# 2. Сборка образа с нужными библиотеками

##### Файлик requirements.txt и его содержимое

In [ ]:
!cat /home/jovyan/requirements.txt

##### Сборка образа с необходимыми библиотеками

In [ ]:
job = client_lib.ImageBuildJob(
                    from_image='registry.aicloud.sbcp.ru/horovod-tf15',
                    requirements_file='/home/jovyan/requirements.txt'
)

In [ ]:
job.submit()

In [ ]:
job.new_image

In [ ]:
job.logs()

# 3. Запуск задачи обучения

##### Создание задачи и отправка на кластер

In [ ]:
mnist_tf_run = client_lib.Job(base_image='registry.aicloud.sbcp.ru/ai0000001-0090/09aa1302-6d95-4f04-8f23-71013679aa00',
                              script='tensorflow_mnist_estimator.py', 
                              n_workers=2, n_gpus=4, warm_cache=False, detached=False)

##### В предыдущей строке мы указали образ, в рамках которого будет исполняться задача (base_image), скрипт, который будет запущен (script), а также количество хостовых исполнителей (n_workers) и количество GPU-карт на каждом (n_gpus). 
##### В этом случае под задачу будет запущено 2 узла по 4 ГПУ карты (ИТОГО - 8 GPU).
##### Теперь вызовем на объекте метод .submit() и отправим задачу в очередь исполнения на кластере

In [ ]:
mnist_tf_run.submit()

